In [1]:
import cvxpy as cp
from cvx.markowitz.backtest import *

%load_ext autoreload
%autoreload 2

In [97]:
markowitz = Markowitz(n_assets=10, n_factors=3)

NONNEGATIVE


In [95]:
markowitz.parameterized_problem.problem.is_dcp()

True

In [42]:
markowitz.parameterized_problem.varia

{'factor_mean': Parameter((3,)),
 'factor_weights': Parameter((3,)),
 'idio_mean': Parameter((10,)),
 'risk_free': Parameter(()),
 'rho_mean': Parameter((10,), nonneg=True),
 'gamma_risk': Parameter((), nonneg=True),
 'factor_covariance_chol': Parameter((3, 3)),
 'idio_vola': Parameter((10,)),
 'volas_times_sqrt_rho': Parameter((10,)),
 'risk_target': Parameter((), nonneg=True),
 'gamma_hold': Parameter((), nonneg=True),
 'kappa_short': Parameter((10,), nonneg=True),
 'kappa_brw': Parameter((), nonneg=True),
 'gamma_trade': Parameter((), nonneg=True),
 'kappa_bas': Parameter((10,), nonneg=True),
 'asset_weights_prev': Parameter((10,)),
 'kappa_mi': Parameter((10,), nonneg=True),
 'gamma_turn': Parameter((), nonneg=True),
 'turnover_limit': Parameter((), nonneg=True),
 'leverage_limit': Parameter((), nonneg=True),
 'weights_lower': Parameter((10,)),
 'weights_upper': Parameter((10,)),
 'cash_lower': Parameter(()),
 'cash_upper': Parameter(())}

In [88]:
import cvxpy as cp

n_assets = 10
n_factors = 3

factor_mean = cp.Parameter(n_factors)
factor_covariance_chol = cp.Parameter((n_factors, n_factors))
idio_mean = cp.Parameter(n_assets)
idio_vola = cp.Parameter(n_assets)
rho_mean = cp.Parameter(n_assets, nonneg=True)
rho_covariance = cp.Parameter(nonneg=True)
risk_free = cp.Parameter()
exposures = cp.Parameter((n_assets, n_factors))
kappa_short = cp.Parameter(n_assets, nonneg=True)
kappa_brw = cp.Parameter(nonneg=True)
kappa_bas = cp.Parameter(n_assets, nonneg=True)
kappa_mi = cp.Parameter(n_assets, nonneg=True)
gamma_risk = cp.Parameter(nonneg=True)
gamma_hold = cp.Parameter(nonneg=True)
gamma_trade = cp.Parameter(nonneg=True)
gamma_turn = cp.Parameter(nonneg=True)
risk_target = cp.Parameter(nonneg=True)
turnover_limit = cp.Parameter(nonneg=True)
leverage_limit = cp.Parameter(nonneg=True)
cash_lower = cp.Parameter(nonneg=True)
cash_upper = cp.Parameter(nonneg=True)
asset_lower = cp.Parameter(n_assets, nonneg=True)
asset_upper = cp.Parameter(n_assets, nonneg=True)
trade_lower = cp.Parameter(n_assets, nonneg=True)
trade_upper = cp.Parameter(n_assets, nonneg=True)


asset_weights = cp.Variable(n_assets)
asset_weights_prev = cp.Parameter(n_assets)
factor_weights = cp.Parameter(n_factors)
cash_weight = cp.Variable()
cash_weight_prev = cp.Parameter()

return_wc = (
    factor_mean @ factor_weights 
    + idio_mean @ asset_weights
    + risk_free * cash_weight 
    - rho_mean @ cp.abs(asset_weights)
)
risk = cp.norm2(
            cp.hstack(
                [
                    cp.norm2(factor_covariance_chol@factor_weights),
                    cp.norm2(cp.multiply(idio_vola, asset_weights)),
                ]
            )
        )
# vola = cp.sum(
#     cp.power(exposures@factor_covariance_chol, 1/2), axis=1) + idio_vola
vola = cp.CallbackParam(callback=lambda:
                                cp.sum(
                                    cp.power(exposures@factor_covariance_chol, 1/2), axis=1) + idio_vola,
                                 shape=asset_weights.shape, 
                                 nonneg=True)
risk_wc = cp.norm2(
            cp.hstack(
                [
                    risk, 
                    rho_covariance**0.5 * vola @ cp.abs(asset_weights)
                    ]
            )
        )

trades = asset_weights - asset_weights_prev
turnover = cp.norm1(trades) 
leverage = cp.norm1(asset_weights)

holding_cost = kappa_short @ cp.pos(-asset_weights) + kappa_brw * cp.pos(-cash_weight)
trading_cost = kappa_bas @ cp.abs(trades) + kappa_mi @ cp.power(cp.abs(trades), 3/2)

objective = (
    return_wc
    - gamma_risk * cp.pos(risk_wc - risk_target)
    - gamma_hold * holding_cost
    - gamma_trade * trading_cost
    - gamma_turn * cp.pos(turnover - turnover_limit)
)

constraints = [
    cp.sum(asset_weights) + cash_weight == 1,    
    cash_weight == cash_weight_prev - cp.sum(trades),
    factor_weights == exposures.T @ asset_weights,
    cash_lower <= cash_weight, cash_weight <= cash_upper,
    asset_lower <= asset_weights, asset_weights <= asset_upper,
    trade_lower <= trades, trades <= trade_upper,
    leverage <= leverage_limit,
]

problem = cp.Problem(cp.Maximize(objective), constraints)
# problem.solve()

In [93]:
vola.sign

'UNKNOWN'

In [92]:
risk_wc.curvature

'UNKNOWN'

In [18]:
a = cp.Parameter(10)
b = cp.Parameter()

c = cp.CallbackParam(shape=10, callback=lambda:
                     a.value*b.value, name="volas_times_sqrt_rho")

c.shape

(10,)

In [ ]:
volas_times_sqrt_rho = cp.CallbackParam(callback=lambda:
                                                variances.value**0.5 * rho_covariance.value**0.5, name="volas_times_sqrt_rho")

In [18]:
a = cp.Parameter()
b = cp.Parameter()

c = a*b

d = cp.CallbackParam(callback=lambda: c.value*b.value)

In [19]:
a.value=3
b.value=4

In [22]:
x = cp.Variable()



Expression(CONVEX, NONNEGATIVE, ())

In [21]:
c.value

12